In [3]:
a = 
a[0].append(0)

print(a)

[[0], [0]]


In [ ]:
from torch.utils.data import DataLoader, IterableDataset
import numpy as np
import webdataset as wb
import os

class ParallelWebDataset(IterableDataset, wb.Composable, wb.Shorthands):
    
    def __init__(self, paths, num_workers):
        self.num_workers = num_workers
        self.paths = [x.tolist() for x in np.array_split(paths, num_workers)]
        self.worker_id = None 
        self.dataset = None
        
        self._processchain = []
        
    def initialise(self, worker_id):
        self.worker_id = worker_id
       
    def __iter__(self):
        print(self.paths[self.worker_id])
        self.dataset = wb.WebDataset(self.paths[self.worker_id])
        self.dataset_iter = iter(self.dataset.map(lambda x: (os.getpid(), x['__key__'], x['__worker__'], x['__url__'])))
        #print(os.getpid(), self.worker_id, self.paths[self.worker_id])
        return self
    
    def __next__(self):
        #print(f"NEXT {os.getpid()}")
        return next(self.dataset_iter)
    
class ParralellWebDataLoader(DataLoader):
    
    def __init__(self, dataset, **kwargs):
        assert isinstance(dataset, ParallelWebDataset)
        kwargs['worker_init_fn'] = dataset.initialise
        kwargs['num_workers'] = dataset.num_workers
        super().__init__(dataset, **kwargs)
        
import glob
import numpy as np
import gymu
import gym_pygame

env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=10)
for i in range(2):
    gymu.data.write_episode(iterator, path=f"Alone-v0/ep-{i}")

episodes = glob.glob("./Alone-v0/*.tar.gz")
dataset = ParallelWebDataset(episodes, num_workers=2)
loader = ParralellWebDataLoader(dataset)

for x in loader:
    print(x)



#for x in loader:
 #   print(x)

In [ ]:
p

In [ ]:
import glob
import numpy as np
import gymu
from gymu.data import Composable
import gym_pygame
import jnu as J

from torch.utils.data import DataLoader
import webdataset as wb

env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=20)
#for i in range(5):
#    gymu.data.write_episode(iterator, path=f"Alone-v0/ep-{i}")

episodes = glob.glob("./Alone-v0/*.tar.gz")
#dataset = wb.WebDataset(episodes, shardshuffle=True)
#dataset = dataset.map(lambda x: (x['__key__'], x['__worker__'], x['__url__'].split("/")[-1].split(".")[0]))

dataset = gymu.data.dataset(episodes)
print(type(dataset).__mro__)
dataset = dataset.gymu.decode(keep_meta=False)
print(type(dataset))

dataset = dataset.gymu.mode(gymu.mode.sa)
dataset = dataset.gymu.to_tensor_dataset(show_progress=True)

loader = DataLoader(dataset, batch_size=2)

for s,a in loader:
    print(s.shape, a.shape)
    #print(x['__key__'], x['__worker__'])

In [ ]:
import glob
import numpy as np
import gymu
from gymu.data import Composable
import gym_pygame
import jnu as J

from torch.utils.data import DataLoader


env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=20)
for i in range(5):
    gymu.data.write_episode(iterator, path=f"Alone-v0/ep-{i}")

episodes = glob.glob("./Alone-v0/*.tar.gz")

dataset = gymu.data.dataset(episodes, shardshuffle=False).then(Composable.decode(keep_meta=True))
#dataset = dataset.shuffle(1000, initial=200)#.map(lambda x: (x['__key__'],))

loader = DataLoader(dataset, num_workers=2)

for x in loader:
    print(x['__key__'], x['__worker__'], x['__url__'])





In [ ]:

from gymu.utils import overload

@overload
def f():
    pass

@f.args(int, int)
def f(x, y):
    print('two integers')

@f.args(float)
def f(x):
    print('one float')

    
print(f.cases)
f(1.)
f(1,1)

In [ ]:
from more_itertools import pairwise


iterator = pairwise([1,2,3,4,5])

x = next(iterator)
for x in iterator:
    print(x)


In [ ]:
import gymu
import jnu as J
import numpy as np

dataset = gymu.data.dataset(["./NORMAL-300k/ep-0000.tar.gz","./NORMAL-300k/ep-0000.tar.gz"]).then(gymu.data.Composable.decode())
dataset = dataset.then(gymu.data.Composable.mode(gymu.mode.sas))

s = [np.concatenate([x.state, (x.next_state if x.next_state is not None else np.zeros_like(x.state))], axis=-1) for x in dataset]
 


    

In [ ]:
J.images(s, scale=3)

In [ ]:
import glob
import numpy as np
import gymu
from gymu.data import Composable
import gym_pygame
import jnu as J


env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=20)
gymu.data.write_episode(iterator, path="Alone-v0/ep-0.tar.gz")
gymu.data.write_episode(iterator, path="Alone-v0/ep-1.tar.gz")

episodes = glob.glob("./Alone-v0/*.tar.gz")
dataset = gymu.data.dataset(episodes).then(Composable.decode())
dataset = dataset.then(gymu.data.Composable.mode(gymu.mode.sas, ignore_last=False))

s = [np.concatenate([x.state, (x.next_state if x.next_state is not None else np.zeros_like(x.state))], axis=-1) for x in dataset]
J.images(s)      








In [ ]:
import jnu as J
import itertools
import gymu
import glob
from gymu.data import Composable


from torch.utils.data import DataLoader



            
            

episodes = glob.glob("./NORMAL-300k/*.tar.gz")

dataset = gymu.data.dataset(episodes)
dataset = dataset.then(Composable.mode(gymu.mode.sa))
dataset = dataset.shuffle(256).batched(256)
loader = DataLoader(dataset, batch_size=None, num_workers=12, prefetch_factor=16)

for (s, a) in loader:
    print(s.shape, a.shape)